Copyright (C) Microsoft Corporation.

# Train with Mask RCNN

First download the model checkpoint from which we will fine tune with our own data.

In [1]:
!wget http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz

--2019-04-18 19:11:15--  http://download.tensorflow.org/models/object_detection/mask_rcnn_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org... 172.217.9.176, 2607:f8b0:4000:814::2010
Connecting to download.tensorflow.org|172.217.9.176|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177817887 (170M) [application/x-tar]
Saving to: ‘mask_rcnn_inception_v2_coco_2018_01_28.tar.gz’

mask_rcnn_inception 100%[===================>] 169.58M   100MB/s    in 1.7s    

2019-04-18 19:11:17 (100 MB/s) - ‘mask_rcnn_inception_v2_coco_2018_01_28.tar.gz’ saved [177817887/177817887]



In [2]:
!tar -xzvf mask_rcnn_inception_v2_coco_2018_01_28.tar.gz

mask_rcnn_inception_v2_coco_2018_01_28/
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.index
mask_rcnn_inception_v2_coco_2018_01_28/checkpoint
mask_rcnn_inception_v2_coco_2018_01_28/pipeline.config
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.data-00000-of-00001
mask_rcnn_inception_v2_coco_2018_01_28/model.ckpt.meta
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/saved_model.pb
mask_rcnn_inception_v2_coco_2018_01_28/saved_model/variables/
mask_rcnn_inception_v2_coco_2018_01_28/frozen_inference_graph.pb


In [15]:
!mkdir model_maskrcnn

We will modify the pipeline.config file, let's first copy it to the folder created above.

In [2]:
!cp ./mask_rcnn_inception_v2_coco_2018_01_28/pipeline.config ./model_maskrcnn/stockout_pipeline.config

Next, we modify the config file to change number of clases to 1 and make other changes as explained [here.](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/configuring_jobs.md). Let's check the final configuration.

In [18]:
!cat ./model_maskrcnn/stockout_pipeline.config

model {
  faster_rcnn {
    number_of_stages: 2
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 800
        max_dimension: 1365
      }
    }
    feature_extractor {
      type: "faster_rcnn_inception_v2"
      first_stage_features_stride: 16
    }
    first_stage_anchor_generator {
      grid_anchor_generator {
        height_stride: 16
        width_stride: 16
        scales: 0.25
        scales: 0.5
        scales: 1.0
        scales: 2.0
        aspect_ratios: 0.25
        aspect_ratios: 0.5
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 3.0
      }
    }
    first_stage_box_predictor_conv_hyperparams {
      op: CONV
      regularizer {
        l2_regularizer {
          weight: 0.0
        }
      }
      initializer {
        truncated_normal_initializer {
          stddev: 0.00999999977648
        }
      }
    }
    first_stage_nms_score_threshold: 0.0
    first_stage_nms_iou_threshold: 0.6
    first

Now, we are ready to train the model. Since training may take a long time (hours), it is adviced to run the following from a  terminal session that remains active during this time. In the terminal, from ./models/research/ directory, run the following commands.

```
conda activate outofstock_env
```

```
export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim
```

```
python object_detection/model_main.py --pipeline_config_path=/datadrive/OutOfStockDemo/model_maskrcnn/stockout_pipeline.config --model_dir=/datadrive/OutOfStockDemo/model_maskrcnn/ --num_train_steps=50000  --sample_1_of_n_eval_examples=10 --alsologtostderr
```


Tensorboard can be run using the following command:

```
tensorboard --logdir=/datadrive/OutOfStockDemo/model_maskrcnn/
```

Tensorflow OD API will report the metrics of your model both on tensorboard and your terminal as the training progresses.

# Export Model

Once the training is done or manually interrupted, you can export your model with the following commands to be used for inferance in the next notebooks.

In [14]:
!mkdir export_maskrcnn

From ./models/research/ directory, run the following by replacing the XXXXX with your model checkpoint.

```
python object_detection/export_inference_graph.py --input_type=image_tensor --pipeline_config_path=/datadrive/OutOfStockDemo/model_maskrcnn/stockout_pipeline.config --trained_checkpoint_prefix=/datadrive/OutOfStockDemo/model_maskrcnn/model.ckpt-XXXXX --output_directory=/datadrive/OutOfStockDemo/export_maskrcnn/
```